In [16]:
import json
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()


True

In [17]:
def geocode(location):
    res = requests.get(f"https://geocode.xyz/{location}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

def getCoords(location):
    try:
        my_coordinates = geocode(location)
        return my_coordinates["coordinates"]
    except:
        return -4.29151, 43.3856

In [18]:
email = os.getenv("EMAIL")
token = os.getenv("TOKEN_SURFCHECK")

def getMoreBeaches(lat, longt):
    url = f"https://surfvideos.xyz/locations.json?latitude={lat}&longitude={longt}&distance=50"

    payload = {}
    headers = {
    'X-User-Email': email,
    'X-User-Token': token,
    'Accept': '{{Accept}}',
    'Content-Type': '{{Content}}'
    }

    response = requests.request("GET", url, headers=headers, data = payload)
    data = response.json()
    
    df = pd.DataFrame(
    [
        data[i]["data"]["attributes"]["name"], 
        data[i]["data"]["attributes"]["address"], 
        data[i]["data"]["attributes"]["direction"], 
        data[i]["data"]["attributes"]["experience"], 
        data[i]["data"]["attributes"]["wave_quality"], 
        data[i]["data"]["attributes"]["frequency"], 
        data[i]["data"]["attributes"]["bottom"], 
        data[i]["data"]["attributes"]["best_wind_direction"], 
        data[i]["data"]["attributes"]["best_swell_direction"],
        data[i]["data"]["attributes"]["latitude"],
        data[i]["data"]["attributes"]["longitude"]
] for i in range(len(data)))
    
    df = df.rename(columns={0: 'Name', 1: 'Address', 2: 'Direction', 3: 'Experience', 4: "Wave quality", 5: "Frequency", 6: 'Bottom', 7: 'Best wind', 8: 'Best swell', 9: "Latitude", 10:"Longitude"})
    df["latlong"] = None
  
    for i in range(df.shape[0]):
        df["latlong"][i] = f"{df['Latitude'][i]},{df['Longitude'][i]}"    
    
    return df


In [19]:
key = os.getenv("KEY")

def getSwellHeight(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellHeight_m"]


def getWindDir(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["winddir16Point"]


def getSwellDir(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellDir16Point"]


def getWindSpeed(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["windspeedKmph"]


def getSwellPeriod(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellPeriod_secs"]


def getWaterTemp(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["waterTemp_C"]


def getWeatherDescription(latlong):
    key = os.getenv("KEY")
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])

    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["weatherDesc"][0]["value"]

In [20]:
def bestBeach(dframe, experience_input):

    cleanup_cols = {"Wave quality_num":     {"Totally Epic": 4, "World Class": 3, "Regional Classic": 2, "Normal": 1, "Sloppy": 0},
                "Frequency_num": {"Very consistent (150 day/year)": 4, "Regular": 3, "Sometimes break": 2, "Rarely break (5day/year)": 1,
                                  "Don't know": 1 }}

    #dframe = dframe[dframe['Wave quality'].notna()]
    #dframe = dframe[dframe['Frequency'].notna()]

    dframe["Wave quality_num"] = dframe["Wave quality"]
    dframe["Frequency_num"] = dframe["Frequency"]

    dframe.replace(cleanup_cols, inplace=True)

    dframe["Score"] = dframe["Wave quality_num"] + dframe["Frequency_num"]
    dframe = dframe.drop(['Wave quality_num', 'Frequency_num'], axis=1)

    dframe = dframe.sort_values(by=['Score'], ascending=False)

    """
    if experience_input == "Beginner":
        new_dframe = dframe[(dframe['Experience'] == "Beginners wave")|(dframe['Experience'] == "All surfers")]
    elif experience_input == "Experienced":
        new_dframe = dframe[(dframe['Experience'] == "All surfers")|(dframe['Experience'] == "Experienced surfers")]
    elif experience_input == "Advanced/Pro":
        new_dframe = dframe[(dframe['Experience'] == "All surfers")|(dframe['Experience'] == "Experienced surfers")|(dframe['Experience'] == "Pros or kamikaze only...")]
        

"""    
    return dframe.head()






In [22]:
user_input = input("Input your current location, if possible enter your exact address:")
experience = input("Input your experience: ")


latlong = getCoords(user_input)
longitude = latlong[0]
latitude = latlong[1]

df = getMoreBeaches(latitude,longitude)
df['Swell Height'] = df['latlong'].apply(getSwellHeight)
df['Swell direction'] = df['latlong'].apply(getSwellDir)
df['Swell period'] = df['latlong'].apply(getSwellPeriod)
df['Wind speed'] = df['latlong'].apply(getWindSpeed)
df['Wind direction'] = df['latlong'].apply(getWindDir)
df['Water temperature'] = df['latlong'].apply(getWaterTemp) 
df['Weather Description'] = df['latlong'].apply(getWeatherDescription)

display(df)
#display(bestBeach(df, experience))

Input your current location, if possible enter your exact address:Comillas, Cantabria, Spain
Input your experience: Experienced


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyError: 'weather'

In [11]:
display(df)

,Name,Address,Direction,Experience,Wave quality,Frequency,Bottom,Best wind,Best swell,Latitude,Longitude,latlong,Swell Height,Swell direction,Swell period,Wind speed,Wind direction,Water temperature,Weather Description
0,Comillas,"Playa de Comillas, Paseo de Jesús Cancio, Semi...",Right,Experienced surfers,Normal,Sometimes break,Reef-rocky,"[SouthWest, South]","[North, NorthWest, West]",43.389955,-4.288316,"43.389955,-4.288316",1.1,WNW,8.8,29,ENE,23,Partly cloudy
1,Oyambre,"Playa de Oyambre, Ría del Capitán-San Vicente ...",Right and left,All surfers,Normal,Regular,Beach-break,"[SouthWest, South, SouthEast]","[NorthWest, West]",43.394633,-4.336081,"43.394633,-4.336081",1.1,WNW,8.8,29,ENE,23,Partly cloudy
2,Meron,"Avenida de Francisco Giner de los Ríos, Barrio...",Right and left,All surfers,Regional Classic,Regular,Beach-break,[SouthEast],"[NorthWest, West]",43.390953,-4.380970,"43.390953,-4.38097",1.1,WNW,8.8,29,ENE,23,Partly cloudy
3,Los Locos,"Playa de los Locos, Calle de Sobremar, Suances...",Right and left,All surfers,Regional Classic,Regular,Beach-break,"[East, NorthEast]","[NorthWest, West]",43.438695,-4.047754,"43.438695,-4.047754",1.1,WNW,8.8,29,ENE,23,Partly cloudy
4,Playa de La Concha,"Playa de la Concha, Calle Doctor Enrique Otí, ...",Right and left,Beginners wave,Choss,Regular,Beach-break,"[SouthWest, South]","[NorthWest, West]",43.436202,-4.041445,"43.436202,-4.041445",1.1,WNW,8.8,29,ENE,23,Partly cloudy
5,Ria de Suances,"Playa de Cuchía, Senda Dunar Cantera Playa de ...",Left,Experienced surfers,Normal,Rarely break (5day/year),Rivermouth,[South],None,43.434855,-4.034697,"43.434855,-4.034697",1.1,WNW,8.8,29,ENE,23,Partly cloudy
6,Los Caballos,"Playa de los Caballos, Calle San Juan, Miengo,...",Right and left,Experienced surfers,Normal,Don't know,Reef-rocky,"[South, SouthEast, East, NorthEast]",[NorthWest],43.444351,-4.023743,"43.444351,-4.023743",1.1,WNW,8.8,29,ENE,23,Partly cloudy
7,Liencres,"Playa de Canallave, Acceso Playa de Valdearena...",Right and left,All surfers,Regional Classic,Regular,Beach-break,"[SouthWest, South, SouthEast, East]","[North, NorthWest]",43.453604,-3.962116,"43.453604,-3.962116",1.1,WNW,8.8,29,ENE,23,Partly cloudy
8,Playa de Vidiago,"Playa de Vidiago, N-634, Llanes, Asturias, Spain",Right and left,All surfers,Normal,Sometimes break,Beach-break,"[South, SouthEast, East, NorthEast]","[North, NorthWest]",43.401384,-4.652174,"43.401384,-4.652174",1.1,WNW,8.9,31,ENE,22,Partly cloudy
9,Covachos,"Tómbolo de Covachos, Barrio La Arnía, Santa Cr...",Right,All surfers,Normal,Regular,Reef-rocky,None,None,43.473726,-3.909309,"43.473726,-3.909309",1.1,WNW,8.8,29,ENE,23,Partly cloudy
